# Fast Fiber Tracking con DiPy

Cargamos los módulos necesarios:

In [1]:
import numpy as np
from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.utils import random_seeds_from_mask
from dipy.reconst.dti import TensorModel
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.direction import peaks_from_model
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import actor, window
from dipy.io.image import save_nifti
from nibabel.streamlines import save as save_trk
from nibabel.streamlines import Tractogram
from dipy.tracking.streamline import Streamlines

Activamos la interactividad.

In [20]:
interactive = True

Cargamos los conjuntos de datos:

In [4]:
fetch_stanford_hardi()
img, gtab = read_stanford_hardi()

data = img.get_data()

Dataset is already in place. If you want to fetch it again please first remove the folder /home/david/.dipy/stanford_hardi 
Dataset is already in place. If you want to fetch it again please first remove the folder /home/david/.dipy/stanford_hardi 


Creamos una máscara:

In [5]:
maskdata, mask = median_otsu(data, 3, 1, False,
                             vol_idx=range(10, 50), dilate=2)

Usando el método de Deconvolusión Restringida Esférica (CSD), estimamos la función respuesta y creamos el modelo:

In [6]:
response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.7)
csd_model = ConstrainedSphericalDeconvModel(gtab, response)

Ajustamos los datos:

In [7]:
sphere = get_sphere('symmetric724')
csd_peaks = peaks_from_model(model=csd_model,
                             data=data,
                             sphere=sphere,
                             mask=mask,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             parallel=True)

Para la tractografía, ajustamos un modelo tensorial:

In [8]:
tensor_model = TensorModel(gtab, fit_method='WLS')
tensor_fit = tensor_model.fit(data, mask)

fa = tensor_fit.fa

Establecemos nuestro discriminante:

In [11]:
tissue_classifier = ThresholdTissueClassifier(fa, 0.1)

Y nuestros puntos semilla dentro de los voxeles:

In [16]:
seeds = random_seeds_from_mask(fa > 0.3, seeds_count=1) # 1 in each voxel

Podemos visualizar una de las rebanadas del campo de direcciones:

In [21]:
ren = window.Renderer()
ren.add(actor.peak_slicer(csd_peaks.peak_dirs,
                          csd_peaks.peak_values,
                          colors=None))

if interactive:
    window.show(ren, size=(900, 900))
else:
    window.record(ren, out_path='csd_direction_field.png', size=(900, 900))

/home/david/anaconda3/envs/DiPyDay/lib/python3.5/site-packages/dipy/viz/colormap.py:233: RuntimeWarning: invalid value encountered in true_divide
  orient = np.abs(orient / np.linalg.norm(orient))


Generamos las líneas de flujo:

In [22]:
streamline_generator = LocalTracking(csd_peaks, tissue_classifier,
                                     seeds, affine=np.eye(4),
                                     step_size=0.5)

streamlines = Streamlines(streamline_generator)

Y las visualizamos:

In [24]:
ren.clear()
ren.add(actor.line(streamlines))

if interactive:
    window.show(ren, size=(900, 900))
else:
    print('Saving illustration as det_streamlines.png')
    window.record(ren, out_path='det_streamlines.png', size=(900, 900))

/home/david/anaconda3/envs/DiPyDay/lib/python3.5/site-packages/dipy/viz/colormap.py:233: RuntimeWarning: invalid value encountered in true_divide
  orient = np.abs(orient / np.linalg.norm(orient))
